<a href="https://colab.research.google.com/github/Dharani1999/Word-embedding-techniques/blob/master/Code_model_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [44]:
import pickle
import gensim
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from gensim.parsing.preprocessing import stem_text
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

from gensim import corpora, models, similarities
from gensim.models import Word2Vec, TfidfModel, LsiModel
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.models.fasttext import FastText

from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorboard as tb
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation,Dense,Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras import optimizers
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.framework import ops

import random
import pathlib
import matplotlib.pyplot as plt

!pip install glove_python
from glove import Corpus, Glove # creating a corpus object

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [45]:
def save_object(obj, filename):
    with open(filename, 'wb') as output:  # Overwrites any existing file.
        pickle.dump(obj, output, pickle.HIGHEST_PROTOCOL)

def load_object(filename):
    with open(filename, 'rb') as input:   # loads an object from a given location
        pickle_object = pickle.load(input)
    return  pickle_object

In [46]:
def pre_process(data):
  _sent=[]
  for i in data:
    sent = word_tokenize(i)
    sent = [w.lower() for w in sent]
    sent = [word for word in sent if word not in stop_words]
    sent = [word for word in sent if word.isalpha()]
    sent = [c for c in sent if not c.isdigit()]
    sent = [stem_text(w) for w in sent]
    #sent = [wordnet_lemmatizer.lemmatize(w) for w in sent]
    _sent.append(sent)
  return _sent

# Give the locations to save the data and word_to_idx_dict
# skip_top skips the given number of most frequent words
# num_words skips words less frequent than given value

from keras.datasets import imdb
data = imdb.load_data(path='/content/drive/My Drive/Movielensdata/ml25m/imdb/data',
    num_words=None,
    skip_top=0,
    maxlen=None,
    seed=113,
    start_char=1,
    oov_char=2,
    index_from=3)

save_object(data, '/content/drive/My Drive/Movielensdata/ml25m/imdb/data')
word_to_idx_dict = imdb.get_word_index(path="/content/drive/My Drive/Movielensdata/ml25m/imdb/imdb_word_index.json")

A local file was found, but it seems to be incomplete or outdated because the auto file hash does not match the original value of 599dadb1135973df5b59232a0e9a887c so we will re-download the data.
17465344/17464789 [==============================] - 0s 0us/step


In [47]:
def data_module(reviews, name, location, output_location, dict_location, tfidf_features=10000, max_seq_length=100, train_data_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent', test_data_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/test_sent', val_data_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/val_sent', Y_train_loc ='/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_train', Y_test_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_test', Y_val_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_val'):
  if reviews==False:
    # Loads ratings.csv file from the given location
    dataset1 = pd.read_csv(location)
    dataset = dataset1.iloc[0:1000,:]
  else:
    # Loads the imdb data from the given location
    dataset = load_object(location)
  if name == 'word2vec':
    data_corpus, users_total = data_word2vec(reviews, dataset, max_seq_length, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'doc2vec':
    data_corpus, users_total = data_doc2vec(reviews, dataset, max_seq_length, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'lsi':
    data_corpus, users_total = data_lsi(reviews, dataset, max_seq_length, dict_loc=dict_location, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'tfidf':
    data_corpus, users_total = data_tfidf(reviews, dataset, max_seq_length, tfidf_features, dict_loc=dict_location, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'glove':
    data_corpus, users_total = data_word2vec(reviews, dataset, max_seq_length, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'hashing':
    data_corpus, users_total = data_hashing(reviews, dataset, max_seq_length, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'cooccur':
    data_corpus, users_total = data_hashing(reviews, dataset, max_seq_length, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  elif name == 'fasttext':
    data_corpus, users_total = data_word2vec(reviews, dataset, max_seq_length, train_loc = train_data_loc, test_loc = test_data_loc, val_loc = val_data_loc, y_train = Y_train_loc, y_test = Y_test_loc, y_val = Y_val_loc)
  
  # Saves the processed train data in the given output location
  save_object(obj=data_corpus, filename=output_location)

def data_word2vec(review,dataset,max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val):
  if review==False:
    dataset.sort_values(by=['userId','timestamp'],inplace=True)
    user_total = len(dataset['userId'].unique())
    
    #Selecting the most recent movies rated by each user and padding if necessary
    movie_list = []
    for i in range(user_total):
      list1 = []
      list1 = dataset.loc[dataset['userId'] ==(i+1),['movieId']]['movieId'].tolist()
      if len(list1)>max_seq_length:
        list1 = list1[(len(list1)-max_seq_length):]
      elif len(list1)<max_seq_length:
        list1 = list1+[0 for j in range((max_seq_length-len(list1)))]
      movie_list.append(list1)

    #Selecting the most recent ratings rated by each user and padding if necessary
    rating_list =[]
    for i in range(user_total):
      list2 = []
      list2 = dataset.loc[dataset['userId'] ==(i+1),['rating']]['rating'].tolist()
      if len(list2)>max_seq_length:
        list2 = list2[(len(list2)-max_seq_length):]
      elif len(list2)<max_seq_length:
        list2 = list2+[0 for j in range((max_seq_length-len(list2)))]
      rating_list.append(list2)

    #Creating user_id level transpose matrices
    movies_transpose = pd.DataFrame(data=movie_list,index=[i+1 for i in range(user_total)])
    movies_transpose.index.names = ['userId']

    ratings_transpose = pd.DataFrame(data=rating_list,index=[i+1 for i in range(user_total)])
    ratings_transpose.index.names = ['userId']

    # Select features from original dataset to form a new dataframe 
    df1 = movies_transpose.iloc[:] # For each row, combine all the columns into one column
    df2 = df1.apply(lambda x: ','.join(x.astype(str)), axis=1) # Store them in a pandas dataframe
    df_clean = pd.DataFrame({'clean': df2}) # Create the list of list format of the custom corpus for gensim modeling 
    sent = [row.split(',') for row in df_clean['clean']]

    # returns list of lists format data and the number of users
    return sent, user_total
  else:
    X_train = dataset[0][0]
    Y_train = dataset[0][1]
    # test-val split of data
    X_test, X_val, Y_test, Y_val = train_test_split(dataset[1][0], dataset[1][1], test_size=0.4, shuffle=False)
    
    #train_seq = X_train.tolist()
    #test_seq = X_test.tolist()
    #val_seq = X_val.tolist()
    #train_sent = pad_sequences(train_seq, padding="post", truncating="post", maxlen=200).tolist()
    #test_sent = pad_sequences(test_seq, padding="post", truncating="post", maxlen=200).tolist()
    #val_sent = pad_sequences(val_seq, padding="post", truncating="post", maxlen=200).tolist()
    
    # Decoding reviews from given integer labels
    reverse_word_index = dict([(value, key) for (key, value) in word_to_idx_dict.items()])
    
    train_reviews=[]
    for j in X_train:
      train_reviews.append(' '.join([reverse_word_index.get(i - 3, '#') for i in j]))

    test_reviews=[]
    for j in X_test:
      test_reviews.append(' '.join([reverse_word_index.get(i - 3, '#') for i in j]))

    val_reviews=[]
    for j in X_val:
      val_reviews.append(' '.join([reverse_word_index.get(i - 3, '#') for i in j]))
    
    # pre-processing train, test, val reviews and saving in given locations
    train_sent=pre_process(train_reviews)
    save_object(obj = train_sent, filename = train_loc)

    test_sent=pre_process(test_reviews)
    save_object(obj = test_sent, filename = test_loc)

    val_sent=pre_process(val_reviews)
    save_object(obj = val_sent, filename = val_loc)

    save_object(obj = Y_train, filename = y_train)
    save_object(obj = Y_test, filename = y_test)
    save_object(obj = Y_val, filename = y_val)

    # returns processed train reviews and the number of train reviews
    return train_sent, len(train_sent)

def data_doc2vec(review, dataset, max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val):
  # Takes the processed train reviews and adds a tag for each review
  Sent, user_total = data_word2vec(review, dataset, max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val)
  tagged_data = []
  tags = []
  for i in range(user_total):
    tagged_data = tagged_data + [TaggedDocument(words=Sent[i], tags=[str(i)])]
  # returns tagged_data and number of users or reviews
  return tagged_data, user_total

def data_lsi(review, dataset, max_seq_length, dict_loc, train_loc, test_loc, val_loc, y_train, y_test, y_val):
  # Takes the processed train reviews and adds a tag for each review
  Sent, user_total = data_word2vec(review, dataset, max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val)

  # Creating a dictionary of all the unique words or tokens
  dictionary = corpora.Dictionary(Sent)

  # Getting a corpus with Bag of Words format and saving the dict in the given location
  corpus = [dictionary.doc2bow(text) for text in Sent]
  dictionary.save(dict_loc)

  return corpus, user_total

def data_tfidf(review, dataset, max_seq_length, max_features, dict_loc, train_loc, test_loc, val_loc, y_train, y_test, y_val):
  # Takes the processed train reviews and adds a tag for each review
  Sent, user_total = data_word2vec(review, dataset, max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val)

  # Creating a dictionary using all the unique tokens or words
  dictionary = corpora.Dictionary(Sent)

  # Filter out tokens that appear in 
  # less than no_below documents,
  # more than no_above documents and
  # keeping only the first keep_n most frequent tokens

  dictionary.filter_extremes(no_below=10, no_above=0.5, keep_n=max_features)

  # Getting a corpus with Bag of Words format and saving the dict in the given location
  corpus = [dictionary.doc2bow(text) for text in Sent]
  dictionary.save(dict_loc)

  return corpus, user_total

def data_hashing(review, dataset, max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val):
  # Takes the processed train reviews and adds a tag for each review
  Sent, user_total = data_word2vec(review, dataset, max_seq_length, train_loc, test_loc, val_loc, y_train, y_test, y_val)

  # Converting list of lists format to list of strings format to suit the input requirement
  corpus = [str(str(doc)[1:-1]) for doc in Sent]
  return corpus, user_total

In [48]:
def embedding_model(name,Data_location,model_save_location,matrix_location, cooccur_features, vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 100,alpha = 0.025,min_alpha=0.00025,dm=1):
  Data = load_object(Data_location)
  if name == 'word2vec':
    word2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers)
  elif name == 'doc2vec':
    doc2vec(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, alpha_=alpha, size_of_window=size_window, no_workers=num_workers, max_epochs=max_num_epochs,min_alpha_=min_alpha, minimum_count=mini_count, dms=dm)
  elif name == 'lsi':
    lsi(input_data=Data, save_loc=model_save_location, total_topics=topics)
  elif name == 'tfidf':
    tfidf(input_data=Data,save_loc=model_save_location)
  elif name == 'glove':
    glove_model(input_data=Data, vec_dims=vector_dims, size_of_window=size_window, save_loc=model_save_location, num_epochs=max_num_epochs, alpha_=0.05, num_threads=4)
  elif name == 'hashing':
    hashing(input_data=Data, vec_dims=vector_dims, save_loc=model_save_location)
  elif name == 'cooccur':
    co_occur(input_data=Data, maximum_features=cooccur_features, save_loc=model_save_location, matrix_loc = matrix_location)
  elif name == 'fasttext':
    fast_text(input_data=Data, save_loc=model_save_location, vec_dims=vector_dims, SG=Sg, size_of_window=size_window, minimum_count=mini_count, no_workers=num_workers, alpha_=0.025)
# doesn't return anything, model is trained and saved in given location

def word2vec(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers):
  # Trains a word2vec model on the training data and saves in the given location
  model = Word2Vec(input_data,min_count=minimum_count,size= vec_dims,workers=no_workers, window =size_of_window, sg = SG)
  model.save(save_loc)

def doc2vec(input_data,save_loc,vec_dims,alpha_,size_of_window,min_alpha_,minimum_count,dms,no_workers,max_epochs):
  # Trains a doc2vec model on the training data and saves in given location
  model = Doc2Vec(size=vec_dims,
                alpha=alpha_, 
                min_alpha=min_alpha_,
                window = size_of_window,
                min_count=minimum_count,
                dm =dms)
  
  model.build_vocab(input_data)
  for epoch in range(max_epochs):
    model.train(input_data, total_examples=model.corpus_count, epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha
  model.save(save_loc)

def lsi(input_data,save_loc,total_topics):
  # Trains an LSI model on the training corpus and saves in the given location
  model = models.LsiModel(corpus=input_data, num_topics=total_topics)
  model.save(save_loc)

def tfidf(input_data,save_loc):
  # Trains a Tf-idf model on the training corpus and saves in the given location
  model = models.TfidfModel(corpus=input_data, normalize=True)
  model.save(save_loc)

def glove_model(input_data,vec_dims,size_of_window,save_loc,alpha_=0.05,num_epochs=30, num_threads=4):
  #training the corpus to generate the co occurence matrix which is used in GloVe
  corpus = Corpus() 
  corpus.fit(input_data, window=size_of_window)

  #creating a Glove object which will use the matrix created in the above lines to create embeddings
  # And saving the model in given location
  glove = Glove(no_components=vec_dims, learning_rate=alpha_) 
  glove.fit(corpus.matrix, epochs=num_epochs, no_threads=4, verbose=True)
  glove.add_dictionary(corpus.dictionary)
  glove.save(save_loc)

def fast_text(input_data,save_loc,vec_dims,SG,size_of_window,minimum_count,no_workers,alpha_=0.025):
  # Trains a fasttext model on the training data and saves in the given location
  model = FastText(min_count=minimum_count, alpha=alpha_, size= vec_dims, workers=no_workers, window =size_of_window)
  model.build_vocab(input_data)
  model.train(input_data, epochs=model.epochs, total_examples=model.corpus_count, total_words=model.corpus_total_words)
  model.save(save_loc)

def hashing(input_data,vec_dims,save_loc):
  # Calls a hashing vectorizer on the training data and saves in the given location
  model = HashingVectorizer(n_features=vec_dims)
  model.transform(input_data)
  save_object(obj=model, filename=save_loc)

def co_occur(input_data, save_loc, matrix_loc, maximum_features):
  # Calling a Count Vectorizer to create a document-term matrix
  model = CountVectorizer(max_df=0.5, ngram_range=(1,1),max_features=maximum_features, token_pattern= r"(?u)\b\w+\b")
  X = model.fit_transform(input_data)

  # Getting the term- term co occurrence matrix from doc-term matrix
  Xc = (X.T * X)
  Xc.setdiag(0)
  # Creating a pandas dataframe with index as the terms, columns as the terms 
  # and each row representing a term's co occurrence frequencies with other terms
  df = pd.DataFrame(data = Xc.toarray(), columns = names, index = names)
  # Saving the model and Dataframe in given locations
  save_object(obj=model, filename=save_loc)
  save_object(obj=df, filename=matrix_loc)


In [49]:
def scoring_module(name,model_loc,data_loc,dict_loc,matrix_location):
  data = load_object(data_loc)
  if name == 'word2vec':
    scores_list = score_word2vec(model_location=model_loc,input_data=data)
  elif name == 'doc2vec':
    scores_list = score_doc2vec(model_location=model_loc,input_data=data)
  elif name == 'lsi':
    scores_list = score_lsi(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'tfidf':
    scores_list = score_tfidf(model_location=model_loc,input_data=data, dict_location=dict_loc)
  elif name == 'glove':
    scores_list = score_glove(model_location=model_loc, input_data=data)
  elif name == 'hashing':
    scores_list = score_hashing(model_location=model_loc, input_data=data)
  elif name == 'cooccur':
    scores_list = score_cooccur(matrix_loc=matrix_location, input_data=data)
  elif name == 'fasttext':
    scores_list = score_fasttext(model_location=model_loc, input_data=data)
  return scores_list

def score_word2vec(model_location,input_data):
  # Loads the trained word2vec model from given location
  model = Word2Vec.load(model_location)
  # Finds embedding for each word and makes a list of all the embeddings
  scored_data = [model.wv[text] for text in input_data]
  return scored_data

def score_doc2vec(model_location,input_data):
  # Loads the trained doc2vec model from given location
  model = Doc2Vec.load(model_location)
  # Finds word embedding for each sentence and makes a list of all
  scored_data = [model.infer_vector(text) for text in input_data]
  return scored_data

def score_glove(model_location,input_data):
  # Loads the trained Glove model from given location
  model = Glove.load(model_location)
  scored_data = []
  # Finds embedding for each word and makes a list of all embeddings
  for doc in input_data:
    list1 = []
    for text in doc:
      list1.append(model.word_vectors[model.dictionary[text]])
    scored_data.append(list1)
  return scored_data

def score_fasttext(model_location, input_data):
  # Loads the trained fasttext model from given location
  model = FastText.load(model_location)
  # Finds embedding for each word and makes a list of all the embeddings
  scored_data = [model.wv[text] for text in input_data]
  return scored_data

def score_lsi(model_location, input_data, dict_location):
  # Loads trained LSI model from given location
  model = LsiModel.load(model_location)
  # Loads the saved dictionary and makes bag of words representation of the data to be scored
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  scored_dat = np.zeros((len(scored_data),len(scored_data[0])))
  # Converts sparse Bag of words embeddings to dense vectors
  for i in range(len(scored_data)):
    for k in range(len(scored_data[i])):
      scored_dat[i][k] = scored_data[i][k][1]
  return scored_dat

def score_tfidf(model_location, input_data, dict_location):
  # Loads trained Tf-idf model from given location
  model = TfidfModel.load(model_location)
  # Loads the saved dictionary and makes bag of words representation of the data to be scored
  dictionary = corpora.Dictionary.load(dict_location)
  scored_data = [model[dictionary.doc2bow(text)] for text in input_data]
  scored_dat = np.zeros((len(scored_data),len(dictionary)), dtype=np.float64)
  # Converts sparse Bag of words embeddings to dense vectors
  for i in range(len(scored_data)):
    for k in range(len(scored_data[i])):
      scored_dat[i][scored_data[i][k][0]] = scored_data[i][k][1]
  return scored_dat

def score_hashing(model_location, input_data):
  # Gets a corpus with list of strings format from the list of lists data
  corpus = [str(str(doc)[1:-1]) for doc in input_data]
  # Loads the saved model from given location
  model = load_object(model_location)
  # the corpus is transformed into sparse vectors
  X = model.transform(corpus)
  Y = X.toarray()
  return Y

def score_cooccur(matrix_loc, input_data):
  # Loads the saved dataframe of term-term co-occurrences
  cooccur_matrix = load_object(matrix_loc)
  # makes a list of word embeddings for each of the word in the data
  scored_data = [cooccur_matrix.loc[text].tolist() for doc in input_data for text in doc]
  return scored_data

In [50]:
def sent_embedding(name,vec_dims,score_list):
  if ((name=='word2vec') or (name=='glove') or (name=='fasttext')):
    # Takes average of word-level embeddings in a sentence to get sentence embedding
    sent_list = []
    for i in range(len(score_list)):
      sum = np.zeros((vec_dims))
      for j in range(len(score_list[i])):
        sum = sum + score_list[i][j]
      sum = sum/(len(score_list[i]))
      sent_list.append(sum)

    # convert to tensor
    sent_tensor = tf.convert_to_tensor(sent_list, dtype=tf.float32)
  elif ((name=='doc2vec') or (name=='tfidf') or (name=='lsi') or (name=='hashing')):
    # For these models, we have sentence level embeddings. Just convert to tensor
    sent_tensor = tf.convert_to_tensor(score_list, dtype=tf.float32)
  return sent_tensor

def Model_build(X,Y,X_predict,Y_predict,accuracy_save_location, sent_save_location,batches_size=100):
  ops.reset_default_graph()

  # The model is trained with dense layers 
  Model = Sequential()
  Model.add(Dense(20, activation='relu'))
  Model.add(Dropout(0.3))
  Model.add(Dense(5, activation='relu'))
  Model.add(Dropout(0.3))
  Model.add(Dense(1, activation='sigmoid'))

  # These hyper-parameters can be changed for a better performance
  opt = optimizers.Adam(lr=0.005, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
  
  Model.compile(loss='binary_crossentropy',optimizer=opt,metrics=['accuracy'])

  # Number of steps in each iteration
  steps_test = int(len(X)/batches_size)
  steps_val = int(len(X_predict)/batches_size)

  # Training the model on scored-test-review-embedding-tensors.
  Model.fit(X, Y, epochs=10, batch_size=batches_size)
  Model.save('/content/drive/My Drive/Movielensdata/ml25m/imdb/model')

  # Finding model predictions on validation data and evaluating with Y_val for accuracy
  sentiment_score = Model.predict(X_predict, batch_size=batches_size, steps=steps_val, workers=3)
  metrics = Model.evaluate(x=X_predict, y=Y_predict, batch_size=batches_size, verbose=1, steps=steps_val, max_queue_size=10, return_dict=False)

  # metrics are loss and accuracy
  accuracy = metrics[1]
  loss = metrics[0]

  # Saving sentiment scores and accuracy in given locations
  save_object(sentiment_score,sent_save_location)
  save_object(accuracy,accuracy_save_location)
  return sentiment_score, accuracy

1) Name can be 'word2vec' or 'doc2vec' or 'lsi' or 'tfidf' or 'glove' or 'cooccur' or 'hashing' or 'fasttext'.

2) location is the path to input file

3) output_location is the path to save the processed data

4) dict_location is the path to dictionary of LSI or TF-IDF

In [51]:
#data_module(reviews=False,name='glove',max_seq_length=100,location='/content/drive/My Drive/Movielensdata/ml25m/ratings.csv',output_location='/content/drive/My Drive/Movielensdata/ml25m/glove/data', dict_location='/content/drive/My Drive/Movielensdata/ml25m/tfidf/dict')

#embedding_model(name='glove',Data_location='/content/drive/My Drive/Movielensdata/ml25m/glove/data',model_save_location='/content/drive/My Drive/Movielensdata/ml25m/glove/glv', matrix_location='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix', vector_dims=10,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 10,alpha = 0.025,min_alpha=0.00025,dm=1,maxi_features=None)

#scoring_input = [['899', '2161', '3949', '5878', '1175', '1237', '8154', '2843', '7365', '4422', '6016'],['1080', '3114', '3671', '2791', '1288', '1', '541', '2692', '7323', '8014', '6370', '4703', '5147']]
#save_object(scoring_input,'/content/drive/My Drive/Movielensdata/ml25m/scoring_input')

#scored_list = scoring_module(name='glove', model_loc='/content/drive/My Drive/Movielensdata/ml25m/glove/glv', data_loc='/content/drive/My Drive/Movielensdata/ml25m/scoring_input', dict_loc='/content/drive/My Drive/Movielensdata/ml25m/tfidf/dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/cooccur/matrix' )

Change name, output location or all models and dict_loc for LSI and TF-IDF 

In [80]:
data_module(reviews=True, name='hashing',tfidf_features =100,max_seq_length=100,location='/content/drive/My Drive/Movielensdata/ml25m/imdb/data',output_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent', dict_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/dict/tfidf_dict', train_data_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent', test_data_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/test_sent', val_data_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/val_sent', Y_train_loc ='/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_train', Y_test_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_test', Y_val_loc = '/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_val')

Change name, Data_location and model_save_location for all models

In [81]:
embedding_model(name='hashing',Data_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/train_sent',model_save_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/trained_model', matrix_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/cooccur_matrix',cooccur_features=1500, vector_dims=100,Sg=1,size_window=3,topics=10,mini_count=1,num_workers=3,max_num_epochs = 10,alpha = 0.025,min_alpha=0.00025,dm=1)

Change name for all models and dict_location for LSI and TF-IDF

In [82]:
scored_test= scoring_module(name='hashing', model_loc='/content/drive/My Drive/Movielensdata/ml25m/imdb/trained_model', data_loc='/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/test_sent', dict_loc='/content/drive/My Drive/Movielensdata/ml25m/imdb/dict/tfidf_dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/imdb/cooccur_matrix' )

In [83]:
scored_val = scoring_module(name='hashing', model_loc='/content/drive/My Drive/Movielensdata/ml25m/imdb/trained_model', data_loc='/content/drive/My Drive/Movielensdata/ml25m/imdb/processed_data/val_sent', dict_loc='/content/drive/My Drive/Movielensdata/ml25m/imdb/dict/tfidf_dict', matrix_location ='/content/drive/My Drive/Movielensdata/ml25m/imdb/cooccur_matrix' )

In [84]:
# Convert word-level embeddings to sentence-level and return tensors
test_tensor = sent_embedding(name='hashing',vec_dims=100,score_list=scored_test)
val_tensor = sent_embedding(name='hashing',vec_dims=100,score_list=scored_val)

# Loads the Y-labels for test and val data
Y_test = load_object('/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_test')
Y_val = load_object('/content/drive/My Drive/Movielensdata/ml25m/imdb/Y/Y_val')

In [85]:
sentiment_scores, accuracy = Model_build(X=test_tensor, Y=Y_test, X_predict=val_tensor, Y_predict=Y_val,accuracy_save_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/accuracy/acc_hashing',sent_save_location='/content/drive/My Drive/Movielensdata/ml25m/imdb/sent_scores/sent_hashing',batches_size=100)

Epoch 1/10
150/150 [==============================] - 0s 2ms/step - loss: 0.6521 - accuracy: 0.6216
Epoch 2/10
150/150 [==============================] - 0s 2ms/step - loss: 0.6074 - accuracy: 0.6757
Epoch 3/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5989 - accuracy: 0.6879
Epoch 4/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5960 - accuracy: 0.6856
Epoch 5/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5884 - accuracy: 0.7005
Epoch 6/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5863 - accuracy: 0.6975
Epoch 7/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5823 - accuracy: 0.7027
Epoch 8/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5765 - accuracy: 0.7049
Epoch 9/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5737 - accuracy: 0.7071
Epoch 10/10
150/150 [==============================] - 0s 2ms/step - loss: 0.5715 - accuracy: 0.7108

In [86]:
sentiment_scores[0:10]

array([[0.6879281 ],
       [0.5491156 ],
       [0.6817043 ],
       [0.64840263],
       [0.03520676],
       [0.6234844 ],
       [0.5138362 ],
       [0.7188653 ],
       [0.5233281 ],
       [0.7415265 ]], dtype=float32)

In [87]:
Y_val[0:10]

array([1, 1, 1, 1, 0, 0, 0, 1, 0, 0])

In [88]:
accuracy

0.6983000040054321